In [2]:
#Imports
import numpy as np
import pandas as pd
import sklearn.linear_model as lm

In [13]:
# Load the data
set_raw = pd.read_csv('../Data/training_set.csv', low_memory=False)

### One Hot encoding and output into a CSV file

In [31]:
seq_raw = toList(set_raw,'Sequence')
chirality_raw = toList(set_raw,'Chirality')
seq_encoded = extract_onehot(seq_raw)

KeyError: 'Chirality'

In [33]:
set_raw.columns

Index(['Sequence', ' Chirality', ' Label'], dtype='object')

### Functions

In [17]:
#One hot encoding function
def onehote(sequence):
    mapping = {"A": 0, "C": 1, "G": 2, "T": 3}
    seq2 = [mapping[i] for i in sequence]
    return np.eye(4,dtype=int)[seq2]

#Converts the dataframe column to a list
def toList(dataframe,column_name):
    return dataframe[column_name].tolist()

#Extracts the one hot encoded data from the sequence list
def extract_onehot(sequence_list):
    return np.array([onehote(i) for i in sequence_list])

#Convert array to dictionary with indexes as keys
def toDictionary(list):
    return {i:list[i] for i in range(len(list))}